In [ ]:
'''
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
'''


'\n!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\nfrom google.colab import auth\nauth.authenticate_user()\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\nimport getpass\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n'

In [ ]:
'''
!mkdir -p drive
!google-drive-ocamlfuse drive
'''

'\n!mkdir -p drive\n!google-drive-ocamlfuse drive\n'

In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [ ]:
'''
!pip install torchvision
'''

'\n!pip install torchvision\n'

In [ ]:
!nvidia-smi

Wed Jul 28 04:53:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install mxnet-cu112 # 没用着呢
#!pip install minpy

In [ ]:
import torch 
x = torch.empty(5,3)
print(x)

tensor([[-2.8933e-22,  3.0824e-41,  3.3631e-44],
        [ 0.0000e+00,         nan,  3.0824e-41],
        [ 1.1578e+27,  1.1362e+30,  7.1547e+22],
        [ 4.5828e+30,  1.2121e+04,  7.1846e+22],
        [ 9.2198e-39,  7.0374e+22, -7.5359e-23]])


# main.py 前面import

In [ ]:
# -*- coding: UTF-8 -*-
# Python
import os
import random

# Torch
import torch
import pandas as pd
import numpy as np  #注意这里可以改用Gpu的Numpy
import torch.nn as nn
import torch.optim as optim
from numpy.core._multiarray_umath import ndarray
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader
from torch.utils import data
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Categorical

# image 
from PIL import Image

# Torchvison
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import CIFAR100, CIFAR10
# from influence import *

# Utils
# import visdom
# from tqdm import tqdm

# Custom
#import models.resnet as resnet
#from config import *
#from data.sampler import SubsetSequentialSampler

# import copy

#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# config.py

In [ ]:
##
# Learning Loss for Active Learning

# Config
NUM_TRAIN = 50000 # N 训练集数据个数
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128  # 128 # 每个dataloader的图片个数
SUBSET    = 25000 # M  # 每次在未标签的数据集中选来聚类的图片个数
ADDENDUM  = 2500  # K  # 每个cycle新加入的不确定度最大的图片个数

MARGIN = 1.0  # xi
WEIGHT = 1.0  # 1.0 # lambda

TRIALS = 1 #尝试 一次
CYCLES = 7  #迭代数

EPOCH = 200  #神经网络训练轮数
LR = 0.1    # 0.1 for SGD # 初始学习速率
MILESTONES = [160] 
EPOCHL = 120 # 

MOMENTUM = 0.9 # optimizer优化器参数之一
WDECAY = 5e-4 # optimizer优化器参数之一

CLUSTER_NUMS = 10 # Kmeans 选取的K值
CLUSTER_MAX_ITER = 10000#聚类最大迭代次数

SEMI_WEIGHT = 0.001

''' CIFAR-10 | ResNet-18 | 93.6%
NUM_TRAIN = 50000 # N
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128 # B
SUBSET    = NUM_TRAIN # M
ADDENDUM  = NUM_TRAIN # K

MARGIN = 1.0 # xi
WEIGHT = 0.0 # lambda

TRIALS = 1
CYCLES = 1

EPOCH = 50
LR = 0.1
MILESTONES = [25, 35]
EPOCHL = 40

MOMENTUM = 0.9
WDECAY = 5e-4
'''


' CIFAR-10 | ResNet-18 | 93.6%\nNUM_TRAIN = 50000 # N\nNUM_VAL   = 50000 - NUM_TRAIN\nBATCH     = 128 # B\nSUBSET    = NUM_TRAIN # M\nADDENDUM  = NUM_TRAIN # K\n\nMARGIN = 1.0 # xi\nWEIGHT = 0.0 # lambda\n\nTRIALS = 1\nCYCLES = 1\n\nEPOCH = 50\nLR = 0.1\nMILESTONES = [25, 35]\nEPOCHL = 40\n\nMOMENTUM = 0.9\nWDECAY = 5e-4\n'

# device设置成cuda或GPU 

In [ ]:
#
# 加一个device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


# sampler.py

In [ ]:
class SubsetSequentialSampler(torch.utils.data.Sampler):
    r"""Samples elements sequentially from a given list of indices, without replacement.

    Arguments:
        indices (sequence): a sequence of indices
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)



# main.py 里面的data

In [ ]:
##
# Data
train_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(size=32, padding=4),
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

test_transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

cifar10_train = CIFAR10('../cifar10', train=True, download=True, transform=train_transform)  # specify data path here
cifar10_unlabeled = CIFAR10('../cifar10', train=True, download=True, transform=test_transform)
cifar10_test = CIFAR10('../cifar10', train=False, download=True, transform=test_transform)

##
# Train Utils
iters = 0



Extracting ../cifar10/cifar-10-python.tar.gz to ../cifar10
Files already downloaded and verified
Files already downloaded and verified


# resnet.py

In [ ]:
#import torch
#import torch.nn as nn
#import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear1 = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out5 = F.avg_pool2d(out4, 4)
        out5 = out5.view(out5.size(0), -1)     # [128, 512]
        out = self.linear(out5)
        out_cons = self.linear1(out5)

        return out, out_cons, out5, [out1, out2, out3, out4]


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet18_student():
    return ResNet(BasicBlock, [1,1,1,1])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])



# main.py 里面的train_epoch函数

In [ ]:
def train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss):
    models['backbone'].train()
    global iters

    for data in dataloaders['train']:
        inputs = data[0].to(device)
        labels = data[1].to(device)
        iters += 1
        # print("train_inputs: ",inputs.shape,"extra_labels: ",labels.shape)

        optimizers['backbone'].zero_grad()

        scores, _, _, features_list = models['backbone'](inputs)
        target_loss = criterion(scores, labels)
        extra_inputs, extra_labels = next(iter(dataloaders['extra']))  # 新加的 这3行 算聚类后伪标签的loss
        # print("extra_inputs_shape: ",extra_inputs.shape,"extra_labels: ",extra_labels)
        extra_inputs = extra_inputs.to(device)
        # 这儿得在外面给extra_dataloader 用Kmeans聚类一下，得到伪标签,加一列标签
        extra_labels = extra_labels.to(device=device, dtype=torch.int64)
        extra_scores, _, _, _ = models['backbone'](extra_inputs)
        extra_loss = criterion(extra_scores, extra_labels)

        backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        aver_extra_loss = torch.sum(extra_loss) / extra_loss.size(0)
        loss = backbone_loss + SEMI_WEIGHT * aver_extra_loss  # 这里的系数也可以改成 0.1试试

        loss.backward()
        optimizers['backbone'].step()


# main.py 里面的test函数

In [ ]:
#
def test(models, dataloaders, mode='val'):
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()

    total = 0
    correct = 0
    with torch.no_grad():
        for (inputs, labels) in dataloaders[mode]:
            #inputs = inputs.cuda()这才是prcharm原来的
            inputs = inputs.to(device)
            #labels = labels.cuda()这才是prcharm原来的
            labels = labels.to(device)
            scores, _, _, _ = models['backbone'](inputs)
            _, preds = torch.max(scores.data, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return 100 * correct / total

# main.py里面的train函数

In [ ]:
#
def train(models, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss, cycle):
    print('>> Train a Model...')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    for epoch in range(num_epochs):

        train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss)
        schedulers['backbone'].step()

        # Save a checkpoint
        if epoch % 20 == 0 or epoch == EPOCH - 1:
            acc = test(models, dataloaders, 'test')
            if best_acc < acc:
                best_acc = acc
                torch.save({
                    'epoch': epoch + 1,
                    'state_dict_backbone': models['backbone'].state_dict()
                    # 'state_dict_module': models['module'].state_dict()
                },
                    '%s/active_resnet18_cifar10.pth' % (checkpoint_dir))
            print('Cycle:', cycle + 1, 'Epoch:', epoch + 1, "---", 'Val Acc: {:.3f} \t Best Acc: {:.3f}'.format(acc, best_acc),flush=True)
            
    print('>> Finished.')


# main.py里面的get_uncertainty函数

In [ ]:
#
def get_uncertainty(models, unlabeled_loader):
    models['backbone'].eval()
    #uncertainty = torch.tensor([]).to(device) #这是原来版本，聚类先不用GPU，用CPU可以用numpy
    uncertainty = torch.tensor([])
    all_features = torch.tensor([]).to(device)

    with torch.no_grad():
        print(">> Start clustering:")

        for (inputs, labels) in unlabeled_loader:
            # inputs = inputs.cuda()这才是pycharm原来的
            inputs = inputs.to(device)
            scores, _, total_feature, features = models['backbone'](inputs)
            # total_feature: [batch_size,512], 行，batch_size，也就是每个batch图片个数，
            # 512是列，是resnet展平的像素feature维度数
            all_features = torch.cat((all_features, total_feature), axis=0)
            # 从未标签数据集中选出的25000个图片的feature

        input_x = np.array(all_features.cpu())
        # input_x: shape: [BATCH_SIZE,512]

        line_num, column_num = input_x.shape  # line_num：样本数量，column_num：每个样本的属性值个数

        cluster_index = np.empty(line_num, dtype=int)  # line_num个样本的聚类结果
        distance = np.empty((line_num, CLUSTER_NUMS), dtype=np.float32)

        # 从line_num个数据样本中不重复地随机选择k个样本作为质心
        cores = input_x[np.random.choice(np.arange(line_num), CLUSTER_NUMS, replace=False)]
        iter = 0

        while True:  # 迭代聚类计算,max=10000次，质心不变时break
            iter = iter + 1
            d = np.square(np.repeat(input_x, CLUSTER_NUMS, axis=0).reshape(line_num, CLUSTER_NUMS, column_num)
                          - cores)
            distance = np.sqrt(np.sum(d, axis=2))  # ndarray(line_num, k)，每个样本距离k个质心的距离，共有line_num行

            index_min = np.argmin(distance, axis=1)  # 每个样本距离最近的质心索引序号 [line_num, 1]
            if (index_min == cluster_index).all() or iter == CLUSTER_MAX_ITER:
                break;

            cluster_index[:] = index_min  # 重新分类 [line_num, 1] 这些图片分别属于哪个cluster, index
            for i in range(CLUSTER_NUMS):  # 遍历质心集
                if len(input_x[cluster_index == i]) != 0:
                    items = input_x[cluster_index == i]  # 找出对应当前质心的子样本集
                    cores[i] = np.mean(items, axis=0)  # 以子样本集的均值作为当前质心的位置

        sorted_distance = np.sort(distance)  # 将矩阵的每一行升序排列

        min_distance_differ = np.abs(sorted_distance[:, 0].reshape(line_num, 1)
                                     - sorted_distance[:, 1].reshape(line_num, 1))

        min_distance_differ_torch = torch.from_numpy(min_distance_differ)

        uncertainty = 10.00 / min_distance_differ_torch

    print(">> Clustering Over:")
    return uncertainty.cpu()

# 创建sudoDataset

In [ ]:
class PseudoDataset(data.Dataset):
    def __init__(self, images, labels, transform):
        self.imgs = images
        self.labels = labels
        self.transforms = transform
    def __getitem__(self, index):
        img_numpy = self.imgs[index]
        if img_numpy.shape[0]==3:
            img_numpy = (np.transpose(img_numpy, (1,2,0)) + 1) / 2.0 * 255.0
        elif img_numpy.shape[0] == 1:
            img_numpy = (img_numpy[0] + 1) / 2.0 * 255.0
        img = Image.fromarray(img_numpy.astype(np.uint8))
        label = np.array(self.labels[index])
        data = self.transforms(img)
        return data, label
    def __len__(self):
        return len(self.imgs)

# 新加的 fakelabel_dataset函数



In [ ]:

#
# extract image list, create new fake label
def fakelabel_dataset(models, unlabeled_loader):
    models['backbone'].eval()
    all_features = torch.tensor([]).to(device)
    # print("cifar10, old_dataset: ", old_dataset)
    all_inputs = torch.tensor([])
    all_labels = torch.tensor([])
    # inputs, labels = old_dataset ## 新加的 是这样吗？取出dataloader里面的label
    total_imgs = np.array([])
    total_labels = np.array([])

    with torch.no_grad():
        print(">> Start clustering:")

        for (inputs, labels) in unlabeled_loader:
            # inputs = inputs.cuda()这才是pycharm原来的
            inputs = inputs.to(device)
            scores, _, total_feature, features = models['backbone'](inputs)
            # total_feature: [batch_size,512], 行，batch_size，也就是每个batch图片个数，
            # 512是列，是resnet展平的像素feature维度数
            all_features = torch.cat((all_features, total_feature), axis=0)
            all_inputs = torch.cat((all_inputs, inputs.cpu()), axis=0)
            all_labels = torch.cat((all_labels, labels.cpu()), axis=0)
            # 从未标签数据集中选出的25000个图片的feature

        input_x = np.array(all_features.cpu())
        total_imgs = np.array(all_inputs.cpu())
        total_labels = np.array(all_labels.cpu())
        # input_x: shape: [BATCH_SIZE,512]

        line_num, column_num = input_x.shape  # line_num：样本数量，column_num：每个样本的属性值个数

        cluster_index = np.empty(line_num, dtype=int)  # line_num个样本的聚类结果
        distance = np.empty((line_num, CLUSTER_NUMS), dtype=np.float32)

        # 从line_num个数据样本中不重复地随机选择k个样本作为质心
        cores = input_x[np.random.choice(np.arange(line_num), CLUSTER_NUMS, replace=False)]
        iter = 0

        while True:  # 迭代聚类计算,max=10000次，质心不变时break
            iter = iter + 1
            d = np.square(np.repeat(input_x, CLUSTER_NUMS, axis=0).reshape(line_num, CLUSTER_NUMS, column_num)
                          - cores)
            distance = np.sqrt(np.sum(d, axis=2))  # ndarray(line_num, k)，每个样本距离k个质心的距离，共有line_num行

            index_min = np.argmin(distance, axis=1)  # 每个样本距离最近的质心索引序号 [line_num, 1]
            if (index_min == cluster_index).all() or iter == CLUSTER_MAX_ITER:
                break;

            cluster_index[:] = index_min  # 重新分类 [line_num, 1] 这些图片分别属于哪个cluster, index
            for i in range(CLUSTER_NUMS):  # 遍历质心集
                if len(input_x[cluster_index == i]) != 0:
                    items = input_x[cluster_index == i]  # 找出对应当前质心的子样本集
                    cores[i] = np.mean(items, axis=0)  # 以子样本集的均值作为当前质心的位置

        sorted_distance = np.sort(distance)  # 将矩阵的每一行升序排列
        min_distance = np.abs(sorted_distance[:, 0].reshape(line_num, 1))  # 每个样本离自己cluster质心的距离

        label_dic = {}  # 新加的，用来匹配原label和真实label
        for i in range(CLUSTER_NUMS):  # 遍历质心集
            if len(input_x[cluster_index == i]) != 0:
                items = input_x[cluster_index == i]  # 找出对应当前质心的子样本集
                item_labels = total_labels[cluster_index == i]
                item_distances = min_distance[cluster_index == i]
                min_index = np.argmin(item_distances)
                label_dic[i] = item_labels[min_index]

        #print(label_dic)
        new_labels = pd.Series(cluster_index).map(label_dic)
        print("cluster_index: ", cluster_index[:200])
        fake_labels = new_labels.values  # 新加的 把cifar_unlabeled 改成假标签的dataset
        #fake_labels = cluster_index

    print(">> Clustering Over:")

    return total_imgs, fake_labels


# main.py 里面的main函数

In [ ]:

##
# Main
if __name__ == '__main__':

    for trial in range(TRIALS):
        # Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
        indices = list(range(NUM_TRAIN))
        random.shuffle(indices)
        START = 2 * ADDENDUM
        labeled_set = indices[:START]
        unlabeled_set = indices[START:]
        semi_subset = unlabeled_set[:SUBSET]  #！！！新加的！！这儿改成了只选一部分数据

        # Model
        #backbone_net = resnet.ResNet18().cuda() #记住这才是原本pycharm里面的
        backbone_net = ResNet18().to(device)
        
        models = {'backbone': backbone_net}

        train_loader = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                  sampler=SubsetRandomSampler(labeled_set),
                                  pin_memory=True)
        
        test_loader = DataLoader(cifar10_test, batch_size=BATCH)
        
        total_loader = DataLoader(cifar10_unlabeled, batch_size=BATCH,
                                          pin_memory=True)
        '''
        j = 0
        for (imgs, labels) in total_loader:
            j+=1
            if(j<5):
                print("initial labels: ", labels) #新加的，检验
        '''
        total_imgs, total_labels = fakelabel_dataset(models, total_loader)
        print("fake_labels: ", total_labels[:100]) #新加的，检验
  
        cifar10_extra = PseudoDataset(total_imgs, total_labels, train_transform)
        
        extra_loader = DataLoader(cifar10_extra, batch_size=BATCH, #新加的
                                  sampler=SubsetSequentialSampler(semi_subset),
                                  pin_memory=True)
        '''
        i = 0
        for (imgs, labels) in extra_loader:
            i+=1
            if(i<5):
                print("extra_loader labels: ", labels) #新加的，检验
        '''
        dataloaders = {'train': train_loader, 'test': test_loader, 'extra': extra_loader}  #新加的

        torch.backends.cudnn.benchmark = True

        # Active learning cycles
        for cycle in range(CYCLES):
            # Loss, criterion and scheduler (re)initialization
            criterion = nn.CrossEntropyLoss(reduction='none')

            optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WDECAY)

            sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)

            optimizers = {'backbone': optim_backbone}
            schedulers = {'backbone': sched_backbone}

            # Training and test
            train(models, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL, cycle)
            acc = test(models, dataloaders, mode='test')
            print('Trial {}/{} || Cycle {}/{} || Label set size {}: Test acc {}'.format(trial + 1, TRIALS, cycle + 1,
                                                                                        CYCLES, len(labeled_set), acc),
                                                                                         flush=True)

            ##
            #  Update the labeled dataset via loss prediction-based uncertainty measurement

            # Randomly sample 10000 unlabeled data points
            random.shuffle(unlabeled_set)
            subset = unlabeled_set[:SUBSET]

            # Create unlabeled dataloader for the unlabeled subset
            unlabeled_loader = DataLoader(cifar10_unlabeled, batch_size=BATCH,
                                          sampler=SubsetSequentialSampler(subset),
                                          # more convenient if we maintain the order of subset
                                          pin_memory=True)

            # Measure uncertainty of each data points in the subset
            uncertainty = get_uncertainty(models, unlabeled_loader)
            #print("main uncertainty: ", uncertainty)
            uncertainty = uncertainty.T
            # Index in ascending order
            arg = np.argsort(uncertainty).numpy().tolist()
            #print("main arg: ", arg) 
            
            # Update the labeled dataset and the unlabeled dataset, respectively
            labeled_set += list(torch.tensor(subset)[arg][-ADDENDUM:].numpy())  # select largest loss
            unlabeled_set = list(torch.tensor(subset)[arg][:-ADDENDUM].numpy()) + unlabeled_set[SUBSET:]

            # labeled_set += list(torch.tensor(subset)[arg][:ADDENDUM].numpy())  # select smallest influence
            # unlabeled_set = list(torch.tensor(subset)[arg][ADDENDUM:].numpy()) + unlabeled_set[SUBSET:]

            # Create a new dataloader for the updated labeled dataset
            dataloaders['train'] = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                              sampler=SubsetRandomSampler(labeled_set),
                                              pin_memory=True)
            
            total_imgs, total_labels = fakelabel_dataset(models, total_loader) #这里又一次聚类
            print("fake_labels: ", total_labels[:100]) #新加的，检验
  
            cifar10_extra = PseudoDataset(total_imgs, total_labels, train_transform)

            extra_subset = unlabeled_set[:SUBSET] #这儿新加的！！
            dataloaders['extra'] = DataLoader(cifar10_extra, batch_size=BATCH,  # BATCH
                                              sampler=SubsetRandomSampler(extra_subset),
                                              pin_memory=True)   #新加的

        # Save a checkpoint
        torch.save({
            'trial': trial + 1,
            'state_dict_backbone': models['backbone'].state_dict()
            # 'state_dict_module': models['module'].state_dict()
        },
            './cifar10/train/weights/active_resnet18_cifar10_trial{}.pth'.format(trial))

        print('---------------------------Current Trial is done-----------------------------',flush=True)


>> Start clustering:
cluster_index:  [5 9 8 6 0 6 0 8 9 6 5 8 9 4 5 9 8 5 0 5 9 4 6 5 5 7 5 6 5 2 3 9 6 0 7 0 7
 5 9 3 5 2 3 0 5 6 0 3 9 1 5 0 6 9 6 8 6 5 3 0 0 7 7 0 0 9 5 8 6 3 3 6 0 6
 7 3 9 5 0 1 3 5 3 7 0 4 5 5 6 7 0 5 9 0 0 2 0 6 5 4 2 0 0 4 4 2 7 5 9 9 7
 7 0 0 0 2 7 3 9 2 7 3 8 9 5 8 9 2 6 7 3 1 5 0 4 7 6 9 9 0 8 3 6 5 9 6 0 0
 8 3 5 3 0 7 9 9 3 5 8 5 1 9 4 6 6 0 0 5 6 0 5 7 8 5 0 5 5 5 6 6 5 5 7 0 6
 2 0 5 2 7 9 1 2 7 9 3 5 3 4 7]
>> Clustering Over:
fake_labels:  [3. 0. 7. 3. 9. 3. 9. 7. 0. 3. 3. 7. 0. 5. 3. 0. 7. 3. 9. 3. 0. 5. 3. 3.
 3. 0. 3. 3. 3. 2. 4. 0. 3. 9. 0. 9. 0. 3. 0. 4. 3. 2. 4. 9. 3. 3. 9. 4.
 0. 9. 3. 9. 3. 0. 3. 7. 3. 3. 4. 9. 9. 0. 0. 9. 9. 0. 3. 7. 3. 4. 4. 3.
 9. 3. 0. 4. 0. 3. 9. 9. 4. 3. 4. 0. 9. 5. 3. 3. 3. 0. 9. 3. 0. 9. 9. 2.
 9. 3. 3. 5.]
>> Train a Model...
Cycle: 1 Epoch: 1 --- Val Acc: 16.860 	 Best Acc: 16.860
Cycle: 1 Epoch: 21 --- Val Acc: 36.360 	 Best Acc: 36.360
Cycle: 1 Epoch: 41 --- Val Acc: 55.270 	 Best Acc: 55.270
Cycle: 1 Epoch: 61 --- 